#### Lab 02 - Collaborative Filtering (User, Film, Rating)

Maintain User Engagement by Generating Recommendation Using Other User Preference

In [1]:
import warnings, pandas

warnings.filterwarnings("ignore")

In [2]:
rating = pandas.read_table("ratings.csv", sep=",")

rating.iloc[:5]

,UserID,MovieID,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
rating.tail()

,UserID,MovieID,Rating,Timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [4]:
# Check Null Value on Rating Table

null = rating.isnull().sum()

null.sum()

0

In [5]:
# Check Average Rating on Rating Table

rata_rata_rating = rating["Rating"].mean()

rata_rata_rating

3.501556983616962

On Average, Our Users Have Given a Rating of 3.5 out of 5.0

In [6]:
# Get Total Film

# Our Total Film is 9.724 Film

total_film = rating["MovieID"].nunique()

total_film

9724

In [7]:
# Get Total User

# Our Total User is 610 User

total_user = rating["UserID"].nunique()

total_user

610

In [8]:
user_rating = rating.groupby("UserID")["Rating"].agg(["count", "mean"])

user_rating.columns = ["Tonton", "Rating"]

user_rating.sort_values("Rating", ascending=False).iloc[:5]

,Tonton,Rating
UserID,,
53,20,5.000000
251,23,4.869565
515,26,4.846154
25,26,4.807692
30,34,4.735294


In [9]:
user_rating = user_rating[user_rating["Tonton"] > 50]

user_rating.sort_values("Rating", ascending=False).iloc[:5]

,Tonton,Rating
UserID,,
523,75,4.693333
348,55,4.672727
171,82,4.634146
452,202,4.556931
43,114,4.552632


In [10]:
film_rating = rating.groupby("MovieID")["Rating"].agg(["count", "mean"])

film_rating.columns = ["Tonton", "Rating"]

film_rating.sort_values("Rating", ascending=False).iloc[:5]

,Tonton,Rating
MovieID,,
88448,1,5.0
100556,1,5.0
143031,1,5.0
143511,1,5.0
143559,1,5.0


In [11]:
film_rating = film_rating[film_rating["Tonton"] > 10]

film_rating.sort_values("Rating", ascending=False).iloc[:5]

,Tonton,Rating
MovieID,,
1041,11,4.590909
3451,11,4.545455
1178,12,4.541667
1104,20,4.475000
2360,12,4.458333


In [12]:
table_film = pandas.read_table("movies.csv", sep=",")

table_film["Genre"] = table_film["Genre"].apply(lambda i : " ".join(i.split("|")))

table_film = table_film[table_film["Genre"] != "(no genres listed)"]

replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  table_film["Genre"] = table_film["Genre"].str.replace(i, t)

table_film.iloc[:5]

,MovieID,Title,Genre
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate = lambda table : round((table.isnull().sum().sum() / table.size) * 100, 3)

SparseRate(rating_table)

98.3

Our 98.3 % Sparsity Rate Signaling Many Things.

* Our Table Mostly Consists of 0

* Our Films Haven't been Watched

* Our Films is Watched, but the User Didn't Rate the Film

* One or Two Films is Watch Less than 10 Times or User Only Watch Less Than 50 Movie

Our Solution is to Set a Minimum Threshold to Film - User Interaction

The First Threshold is Film. Our Minimum Threshold for a Film is

*This Film Must be Watched, Minimum of 10 Times*

In [14]:
minimum_film_interaction = 10

WatchFilm = rating.groupby("MovieID")["Rating"].agg(["count", "mean"])

WatchFilm.columns = ["TotalWatch", "Rating"]

WatchFilm = WatchFilm[WatchFilm["TotalWatch"] > minimum_film_interaction]

rating = rating[rating["MovieID"].isin(WatchFilm.index)]

rating["MovieID"].nunique()

2121

In [15]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

93.845

The Last Threshold is User. Our Minimum Threshold for a User is

*This User Must be Watch & Rate, Minimum of 50 Films*

In [16]:
minimum_user_interaction = 50

UserWatch = rating.groupby("UserID")[["Rating"]].count()

UserWatch = UserWatch[UserWatch["Rating"] > minimum_user_interaction]

rating = rating[rating["UserID"].isin(UserWatch.index)]

In [17]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

90.396

Our Final Sparsity Rate is 90.39 %

In [18]:
rating = rating.join(table_film.set_index("MovieID"), on="MovieID")

rating = rating.drop("Timestamp", axis="columns")

rating.iloc[:5]

,UserID,MovieID,Rating,Title,Genre
0,1,1,4.0,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy Romance
2,1,6,4.0,Heat (1995),Action Crime Thriller
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery Thriller
4,1,50,5.0,"Usual Suspects, The (1995)",Crime Mystery Thriller


In [19]:
rating_table.iloc[:5, :8]

UserID,1,4,6,7,10,11,15,16
MovieID,,,,,,,,
1,4.0,NaN,NaN,4.5,NaN,NaN,2.5,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN


In [20]:
rating_table = rating_table.fillna(0)

rating_table.iloc[:5, :8]

UserID,1,4,6,7,10,11,15,16
MovieID,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0


What is Sparse Matrix ? Sparse Matrix is Matrix that Many of the Value is 0

Usually Sparse Matrix is Mostly Use in Collaborative Method Recommendation System

In [21]:
from scipy.sparse import csr_matrix

rating_matrix = csr_matrix(rating_table.values)

rating_matrix.shape

(2121, 352)

In [22]:
rating_reset = rating_table.reset_index()

rating_reset.shape

(2121, 353)

In [23]:
rating_reset.iloc[:5, :10]

UserID,MovieID,1,4,6,7,10,11,15,16,17
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0


In [24]:
rating_reset.columns.name

'UserID'

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric="cosine", algorithm="brute")

model.fit(rating_matrix)

## `Main Model`

### 🕵 Challenges

* Find Similar Movie to `Avengers, The (2012)`

* Find Similar Movie to `Guardians of the Galaxy (2014)`

* Find Similar Movie to `Forrest Gump (1994)`

* Find Similar Movie to `Toy Story (1995)`

In [26]:
input_title = "Avengers"

movies_index = table_film[table_film["Title"].str.contains(input_title)]

movies_index = movies_index.iloc[2]['MovieID']

table_film[table_film["MovieID"] == movies_index]

,MovieID,Title,Genre
7693,89745,"Avengers, The (2012)",Action Adventure SciFi IMAX


In [27]:
rating[rating["MovieID"] == movies_index].iloc[:8]

,UserID,MovieID,Rating,Title,Genre
1535,15,89745,2.0,"Avengers, The (2012)",Action Adventure SciFi IMAX
2182,18,89745,4.0,"Avengers, The (2012)",Action Adventure SciFi IMAX
3520,21,89745,4.0,"Avengers, The (2012)",Action Adventure SciFi IMAX
7891,52,89745,5.0,"Avengers, The (2012)",Action Adventure SciFi IMAX
9030,62,89745,4.0,"Avengers, The (2012)",Action Adventure SciFi IMAX
9400,63,89745,3.5,"Avengers, The (2012)",Action Adventure SciFi IMAX
11555,68,89745,4.5,"Avengers, The (2012)",Action Adventure SciFi IMAX
11950,73,89745,4.0,"Avengers, The (2012)",Action Adventure SciFi IMAX


In [28]:
movies_index = rating_reset[rating_reset["MovieID"] == movies_index]

movies_index = movies_index.index[0]

movies_index

1961

In [29]:
rating_reset.iloc[1960:1965, :10]

UserID,MovieID,1,4,6,7,10,11,15,16,17
1960,89492,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1961,89745,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1962,89774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1963,89864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1964,89904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
total_limit = 10

total_limit += 1

distances, table_indexes = model.kneighbors(rating_matrix[movies_index], n_neighbors=total_limit)

sorter = list(zip(table_indexes.flatten().tolist(), distances.flatten().tolist()))

table_indexes = sorted(sorter, key=lambda x: x[1])

table_indexes[:5]

[(1961, 0.0),
 (2060, 0.23810542775945065),
 (1821, 0.2771270484831855),
 (1907, 0.29182116047789386),
 (1970, 0.2938930732962426)]

In [31]:
molist = [i[0] for i in table_indexes]

molist[:5]

[1961, 2060, 1821, 1907, 1970]

In [32]:
molist = [rating_reset.iloc[i, 0] for i in molist]

molist[:5]

[89745, 112852, 59315, 77561, 91529]

In [33]:
table_film[table_film["MovieID"].isin(molist)]

,MovieID,Title,Genre
6743,59315,Iron Man (2008),Action Adventure SciFi
7324,77561,Iron Man 2 (2010),Action Adventure SciFi Thriller IMAX
7372,79132,Inception (2010),Action Crime Drama Mystery SciFi Thriller IMAX
7620,87232,X-Men: First Class (2011),Action Adventure SciFi Thriller War
7693,89745,"Avengers, The (2012)",Action Adventure SciFi IMAX
7768,91529,"Dark Knight Rises, The (2012)",Action Adventure Crime IMAX
8151,102125,Iron Man 3 (2013),Action SciFi Thriller IMAX
8395,110102,Captain America: The Winter Soldier (2014),Action Adventure SciFi IMAX
8425,111362,X-Men: Days of Future Past (2014),Action Adventure SciFi
8475,112852,Guardians of the Galaxy (2014),Action Adventure SciFi


In [34]:
def OutputFilm(title, limit=5):

  movies_index = table_film[table_film["Title"] == title]
  movies_index = movies_index["MovieID"].values[0]
  movies_index = rating_reset[rating_reset["MovieID"] == movies_index].index[0]

  limit += 1

  distances, table_indexes = model.kneighbors(rating_matrix[movies_index], n_neighbors=limit)
  sorter = list(zip(table_indexes.flatten().tolist(), distances.flatten().tolist()))
  table_indexes = sorted(sorter, key=lambda x: x[1])

  molist = [i[0] for i in table_indexes]

  molist = [i for i in rating_reset.iloc[molist, 0].values][1:]
  result = table_film[table_film["MovieID"].isin(molist)]
  return result

title = "Guardians of the Galaxy (2014)"

OutputFilm(title, 15)

,MovieID,Title,Genre
6743,59315,Iron Man (2008),Action Adventure SciFi
7324,77561,Iron Man 2 (2010),Action Adventure SciFi Thriller IMAX
7589,86332,Thor (2011),Action Adventure Drama Fantasy IMAX
7620,87232,X-Men: First Class (2011),Action Adventure SciFi Thriller War
7646,88140,Captain America: The First Avenger (2011),Action Adventure SciFi Thriller War
7693,89745,"Avengers, The (2012)",Action Adventure SciFi IMAX
8395,110102,Captain America: The Winter Soldier (2014),Action Adventure SciFi IMAX
8425,111362,X-Men: Days of Future Past (2014),Action Adventure SciFi
8438,111759,Edge of Tomorrow (2014),Action SciFi IMAX
8683,122886,Star Wars: Episode VII - The Force Awakens (2015),Action Adventure Fantasy SciFi IMAX


In [35]:
title = "Forrest Gump (1994)"

OutputFilm(title, 5)

,MovieID,Title,Genre
97,110,Braveheart (1995),Action Drama War
257,296,Pulp Fiction (1994),Comedy Crime Drama Thriller
277,318,"Shawshank Redemption, The (1994)",Crime Drama
418,480,Jurassic Park (1993),Action Adventure SciFi Thriller
1939,2571,"Matrix, The (1999)",Action SciFi Thriller


In [36]:
title = "Toy Story (1995)"

OutputFilm(title, 8)

,MovieID,Title,Genre
123,150,Apollo 13 (1995),Adventure Drama IMAX
224,260,Star Wars: Episode IV - A New Hope (1977),Action Adventure SciFi
257,296,Pulp Fiction (1994),Comedy Crime Drama Thriller
314,356,Forrest Gump (1994),Comedy Drama Romance War
418,480,Jurassic Park (1993),Action Adventure SciFi Thriller
911,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action Adventure SciFi
2355,3114,Toy Story 2 (1999),Adventure Animation Children Comedy Fantasy
3194,4306,Shrek (2001),Adventure Animation Children Comedy Fantasy Ro...
